# Assignment 4.  
## Programming simple SQL queries from a host Programming Language (Python)

In this assigment, we will fetch data from a relational database server directly into Python variables. Using these new skills, we will complete the problems from Chapters 5 and 6 in *SQL Queries for Mere Mortals*.

### Connect to the database 

First, let's assign the login credentials. We will use the python libary `getpass` to enter the password without displaying it.

In [3]:
import getpass

host = "db.data-science-ust.net"
user = "dimitri"    # !! Replace with your user name !!
password = getpass.getpass()

········


Then we will use the **MySQL client library** called `pymysql`. Client libraries provide all the functionality to interact with the database from the client side.

In [4]:
import pymysql

# establish a database connection
conn = pymysql.connect(host=host, user=user, passwd=password)

#### Execute an SQL query

Using the connection object `conn`, we obtain a `cursor`: an object that manages the execution of queries. Cursors can be configured in various ways. This code constructs a cursor that retrieves query output in the form of lists of Python `dict`s.

In [5]:
cursor = conn.cursor(cursor=pymysql.cursors.DictCursor)

Now we can use `cursor` to send queries to the server and retrieve the result. The `execute` method returns the number of items in the query result but the data themselves are still inside `cursor`.

In [ ]:
sql = """
SELECT * 
    FROM shared_sales.customers
    ORDER BY CustLastName, CustFirstName
    LIMIT 10
"""
cursor.execute(sql)

#### Fetch items (database rows) 
To fetch the next item from the result as `dict`, use the `cursor.fetchone()`. The keys of the dict correspond to column names with row values in the values of the dict.

In [ ]:
first_item = cursor.fetchone()
print(first_item)

Call it again to fetch the next item

In [ ]:
second_item = cursor.fetchone()
print(second_item)

To fetch all the remaining items as a `list` of `dict`s, use `cursor.fetchall()`

In [ ]:
items = cursor.fetchall()
items

Note that running `execute.fetchall()` the second time yields an empty list: we got the entire result set out of the `cursor`!

In [ ]:
cursor.fetchall()

And `cursor.fetchone()` returns `None`.

In [ ]:
cursor.fetchone()

You must execute a new query to fill the `cursor` with new contents.

In [ ]:
cursor.execute(sql)

In addition to the two `fetch` methods, you can use the `cursor` as the generator in a `for` loop to iterate over the elements of the result set:

In [ ]:
for item in cursor:
    print("{CustomerID}: {CustLastName}, {CustFirstName}".format(**item))

# Examples

#### Example 1
What is the inventory value of each product (price x quantities)?

In [ ]:
cursor.execute("USE shared_sales")  # enter the right database

In [ ]:
# assign the SQL query string to a variable to use in a query later
sql = """
SELECT ProductNumber, ProductName, QuantityOnHand * RetailPrice as InventoryValue
FROM products
"""

In [ ]:
# execute the SQL query
cursor.execute(sql)

Now retrieve and show the result:

In [ ]:
results = cursor.fetchall()
print(results)

You may use the `pprint` module from the [standard library](https://docs.python.org/3/library/pprint.html) to pretty-print the resulting dict list:

In [ ]:
import pprint
pprint.pprint(results)

Or you may like to convert it into a pandas dataframe of display and manipulation.

In [ ]:
import pandas as pd
pd.DataFrame.from_records(results, index=["ProductNumber"])

This is enough to fulfill the requirement of the assignment but you may use your Python string formatting skills to produce a nicer output or to perform additional operations.

In [ ]:
# Execute query again
cursor.execute(sql)

# print heading
item_line = "{ProductNumber:3}: {ProductName:40} {InventoryValue:>8}" 
heading = item_line.format(ProductNumber="#", ProductName="Product Name", InventoryValue="Value")
print(heading)
print('-' * len(heading)) # dividing line

# print items from cursor
total = 0
for item in cursor:
    print(item_line.format(**item))
    total += item['InventoryValue']

# print total    
print('-' * len(heading))
print(item_line.format(ProductNumber='', ProductName='TOTAL:', InventoryValue=total))

# Assignment Problems

Note that table names and column names may differ from those in the book. Use `SHOW DATABASES`, `USE <database>`, and `SHOW TABLES` commands to navigate the databases and `SHOW CREATE TABLE <table>` to inspect the declaration of the individual tables.

## In database `shared_sales`.

#### Problem 1 .

Show all products with price reduced by 5%.

#### Problem 2.
Show the list of orders made by each customer in descending order date. Hint: you might need to order by more than one column.

#### Problem 3.
Show vendor name and addresses sorted by vendor name.

## In database `shared_entertain`

#### Problem 4.
Show all customers ordered by city.

#### Problem 5.
List all entertainers and their Web sites.

#### Problem 6.
Show the date of each agent's first six-month performance review.


**Hint:** You will need date arithmetic https://dev.mysql.com/doc/refman/5.7/en/date-and-time-functions.html

## In database `shared_schedule`

#### Problem 7
Show the staff members in descending order of salary.

#### Problem 8
Make the staff phone list.

#### Problem 9
List students ordered by the city they live in.

## In database `shared_bowling`.

#### 10. Show next year's tournament date for each tournament location.

**Hint:** Add 364 days to get the same day of the week. See date and time functions here: https://dev.mysql.com/doc/refman/5.7/en/date-and-time-functions.html

In [15]:
cursor.execute('USE shared_bowling')

0

In [24]:
cursor.execute(
    """
    SELECT *, ADDDATE(TourneyDate, INTERVAL 364 DAY) as NextYearTourney
    FROM Tournaments 
    ORDER BY TourneyLocation
    """)

20

In [25]:
cursor.fetchone()

{'TourneyID': 7,
 'TourneyDate': datetime.date(2017, 10, 16),
 'TourneyLocation': 'Acapulco Lanes',
 'NextYearTourney': datetime.date(2018, 10, 15)}

#### 11. List names and phone numbers of the members of the league 
(32 rows)

#### 12. List each team's lineup.
(32 rows)

## In database `shared_sales`

#### 13. List vendors from the cities of Ballard, Bellevue, and Redmond.
(3 rows)

#### 14. Show alphebetized list of products with a retail price of \$1,250.00 or more
(13 rows)

#### 15. List vendors with no websites

## In database `shared_entertain`

#### 16. List engagements that occurred during October 2017
(24 rows)

#### 17. Show engagements in October 2017 that start between noon and 5 pm.

#### 18. Show engagements that start and end on the same day.

## In database `shared_schedule`

#### 19. Show staff members that use a PO Box as their address.

#### 20. Show students who live outside the Pacific Northwest

#### 21. Show subjects with subject code starting with "MUS"

Please review string comparison functions https://dev.mysql.com/doc/refman/5.7/en/string-comparison-functions.html

#### 22. List the ID numbers of all Associate professors who are employed full time.

## In database `shared_bowling`

#### 23. List tournaments held in 2017

#### 24. List tournament schedules for Bolero, Red Rooster, and Thunderbird Lanes.

#### 25. List the bowlers who live in Bellevue, Bothell, Duvall, Redmond, and Woodinville who are on teams 5, 6, 7, or 8.

## In database `shared_recipes`

#### 26. List all recipes for main courses (recipe class 1) that have notes.

#### 27. Display the first five recipes.